In [7]:
import pybamm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
model = pybamm.lithium_ion.DFN(options={"calculate discharge energy":"true"})
for event in model.events: 
    if event.name == "Zero electrolyte concentration cut-off":
        model.events.remove(event)
chem = pybamm.parameter_sets.Chen2020 #Cell Chemistry
#chem 

In [ ]:
Parameter_values = pybamm.ParameterValues(chemistry=chem)
df = pd.DataFrame()
# Set number of var_pts for the mesh
var = pybamm.standard_spatial_vars
var_pts = {var.x_n: 20, var.x_s: 20, var.x_p:400, var.r_n: 150, var.r_p: 20}
C_rates = (1,2)
sims = []
Energy = []
Power = []
Factor = []
param_name = "Initial concentration in electrolyte [mol.m-3]"
param_list = (0.5,1)
for factor in param_list: 
     sims = []
     Parameter_values = pybamm.ParameterValues(chemistry=chem)
     Parameter_values[param_name] *= factor
     for C_rate in C_rates:
     #Parameter_values["Current function [A]"] = 5 * C_rate
     #sim = pybamm.Simulation(model, parameter_values=Parameter_values)
     #sim.solve([0, 4000 / C_rate])
     #sims.append(sim)
          if C_rate < 2.1:
               t_eval = np.linspace(0, 4000 / C_rate, 500)
               dt_max = None
          elif C_rate < 4:
               t_eval = np.linspace(0, 3500 / C_rate, 1000)
               dt_max = 0.0001
          elif C_rate < 8:
               t_eval = np.linspace(0, 3000 / C_rate, 500)
               dt_max = 0.0001
          else:
               t_eval = np.linspace(0, 60, 500)
               dt_max = 0.0001
          solver = pybamm.CasadiSolver(mode="safe", dt_max=dt_max, max_step_decrease_count=15)
          power = C_rate*15 
          experiment = pybamm.Experiment([f"Discharge at {power} W until 2.5 V"], period="40 seconds")
          sim = pybamm.Simulation(model, parameter_values=Parameter_values, var_pts=var_pts, solver=solver, C_rate=C_rate, experiment=experiment)
          sim.solve()
          sims.append(sim)
          print(sim.solution.termination)
          Energy.append(sol.solution["Discharge energy [W.h]"].data[-1] for sol in sims)
          Power.append(sol.solution["Power [W]"].data[-1] for sol in sims)
          Factor.append(factor)
     dict = {'Energy': Energy, 'Power': Power, 'Factor': Factor }
     df = pd.DataFrame(dict)
    

In [9]:
df

,Energy,Power,Factor
0,<generator object <genexpr> at 0x000001D83E970...,<generator object <genexpr> at 0x000001D83E970...,0.5
1,<generator object <genexpr> at 0x000001D83EDB4...,<generator object <genexpr> at 0x000001D83EDB4...,0.5
2,<generator object <genexpr> at 0x000001D83EB5C...,<generator object <genexpr> at 0x000001D83EB5C...,1.0
3,<generator object <genexpr> at 0x000001D84358C...,<generator object <genexpr> at 0x000001D84358C...,1.0


In [10]:
Energy

[<generator object <genexpr> at 0x000001D83E970F20>,
 <generator object <genexpr> at 0x000001D83EDB4900>,
 <generator object <genexpr> at 0x000001D83EB5CB30>,
 <generator object <genexpr> at 0x000001D84358C4A0>]

In [11]:
Power

[<generator object <genexpr> at 0x000001D83E970F90>,
 <generator object <genexpr> at 0x000001D83EDB4970>,
 <generator object <genexpr> at 0x000001D83EB5CAC0>,
 <generator object <genexpr> at 0x000001D84358C5F0>]

In [12]:
Factor

[0.5, 0.5, 1, 1]